In [42]:

state = [input() for _ in range(8)]
line_1 = [0,1,0,1,0,1,0,1]
line_2 = [1,0,1,0,1,0,1,0]
chess = []
for i in range(8):
    if i % 2 == 0:
        chess.append(line_1)
    else:
        chess.append(line_2)
for x in state:
    for y in x:


7


In [33]:
m = int(input())
n = int(input())


10
35


In [9]:
n = input()
length = len(n)
total = 0
def change(n):
    if n in "0123456789":
        return int(n)
    else:
        return ord(n) - 55
for i in range(length):
    digit = n[-(i+1)]
print(sum(total))

10


#### 확인

In [6]:
from db.db import upload_contents
if __name__ == "__main__":
    # JSON 파일 경로와 컬렉션 이름만 지정해주면 됩니다
    data = upload_contents(
        json_path=r"C:\PythonEnvs\huggingface\OPIc_Buddy\data\opic_question.json",
        collection_name="opic_samples",
        overwrite=True
    )
    print(type(data))

MongoDB 연결 성공 (컬렉션: opic_samples)
Inserted 79 documents into OPIcBuddy.opic_samples
<class 'NoneType'>


In [7]:
# run_embedding.py
from embedding import embedding_collection

if __name__ == "__main__":
    MODEL_PATH = r"C:\PythonEnvs\huggingface\OPIc_Buddy\model\models--intfloat--e5-base-v2\snapshots\f52bf8ec8c7124536f0efb74aca902b2995e5bcd"

    embedding_collection(
        input_collection="opic_samples",
        output_collection="embedded_opic_samples",
        model_path=MODEL_PATH,
        batch_size=50
    )
    print("임베딩 완료!")

MongoDB 연결 성공 (컬렉션: opic_samples)
MongoDB 연결 성공 (컬렉션: embedded_opic_samples)


Embedding docs: 79it [00:00, 179.60it/s]


✅ 임베딩 완료!


In [4]:
from rag import run_rag

EMBED_MODEL_PATH = r"C:\PythonEnvs\huggingface\OPIc_Buddy\model\models--intfloat--e5-base-v2\snapshots\f52bf8ec8c7124536f0efb74aca902b2995e5bcd"
resp = run_rag("yoga", model_path=EMBED_MODEL_PATH, k=3)
print(resp)


MongoDB 연결 성공 (컬렉션: embedded_opic_samples)
You are an OPIc speaking test assistant.
Use the following context to ask ONE natural follow-up question in English.

Context:
How often do you do yoga? Where do you usually do it, and what do you like most about it?
What do you like most about yoga? Do you find it helps with your physical flexibility, or your mental focus?
Tell me about a memorable yoga experience you've had. What happened, and how did you feel after the session?

User said: yoga
Your follow-up question:

How often do you do yoga?

Answer: I do yoga three times a week. I usually do it at home, in my living room. I like to do it in the morning, before I start my day. I find it helps me to relax and calm my mind.

How often do you do yoga?

Answer: I do yoga three times a week. I usually do it at home, in my living room. I like to do it in the morning, before I start my day. I find it helps me to relax and calm my mind.

How often do you do yoga?

Answer: I do yoga three times 

In [10]:
import random
from typing import List, Optional
from retriever import retrieve_top_k

EMBED_MODEL_PATH = r"C:\PythonEnvs\huggingface\OPIc_Buddy\model\models--intfloat--e5-base-v2\snapshots\f52bf8ec8c7124536f0efb74aca902b2995e5bcd"

def get_random_questions(
    user_input: str,
    k: int = 3,                                # 유사 문서(컨텍스트) 몇 개 가져올지
    collection_name: str = "embedded_opic_samples",
    model_path: str = EMBED_MODEL_PATH,
    text_field: str = "content",
    emb_field: str = "embedding",
    category: Optional[str] = None,
    topic: Optional[str] = None,
    max_pick: int = 3                          # 최종 무작위 질문 개수 상한(1~max_pick)
) -> List[str]:
    # 1) retrieval (필수 인자 model_path 전달!)
    contexts = retrieve_top_k(
        query=user_input,
        model_path=model_path,
        collection_name=collection_name,
        k=k,
        text_field=text_field,
        emb_field=emb_field,
        category=category,
        topic=topic
    )  # -> List[str]

    # 2) 줄 단위로 분해 & 정리
    #    (컨텍스트 한 개에 질문 여러 줄이 들어있는 경우가 많음)
    lines: List[str] = []
    for ctx in contexts:
        if not ctx:
            continue
        for line in str(ctx).splitlines():
            s = line.strip()
            if s:
                lines.append(s)

    # 중복 제거(순서 보존)
    seen = set()
    questions = []
    for q in lines:
        if q not in seen:
            seen.add(q)
            questions.append(q)

    if not questions:
        return ["질문을 찾을 수 없습니다."]

    # 3) 1~max_pick개 무작위 선택
    num_to_pick = random.randint(1, min(max_pick, len(questions)))
    return random.sample(questions, num_to_pick)

# 예시 실행
if __name__ == "__main__":
    user_answer = "I usually go hiking on weekends."
    follow_ups = get_random_questions(
        user_answer,
        k=1,                                # 유사 문서 3개 가져와서
        collection_name="embedded_opic_samples"  # 실제 사용하는 컬렉션명으로
    )
    for q in follow_ups:
        print(q)


MongoDB 연결 성공 (컬렉션: embedded_opic_samples)
How often do you go hiking or trekking? Where do you usually go, and what do you like most about it?


In [11]:
from quest import sample_opic_test

if __name__ == "__main__":
    survey_keys = [
        "dormitory", "student", "driving", "travel", "hobby",
        "food", "family", "friend", "neighborhood", "shopping",
        "sports", "health", "movie", "music", "part-time job"
    ]
    qs = sample_opic_test(
        user_input="travel and student life",
        survey_keys=survey_keys,
        k_retrieve=400,
        seed=123,
        collection_name="embedded_opic_samples"
    )
    for i, q in enumerate(qs, start=1):
        print(f"{i}. {q}")

MongoDB 연결 성공 (컬렉션: embedded_opic_samples)
1. Please introduce yourself. Tell me your name, where you’re from, and a few things about your background.
2. What is your favorite thing to buy when you go shopping? Describe your shopping style and what you look for when you buy it.
3. I'd like to give you a situation for you to act out. You want to go shopping with your friend. Ask 3-4 questions to your friend.
4. Tell me about your first driving experience. Who taught you, and how did you feel when you were behind the wheel for the first time?
5. How often do you visit a spa or massage shop, and what benefits do you get from it? Describe your routine and how it affects your health.
6. Tell me about the last time you went to a club. Who did you go with, what did you do, and what was the music like?
7. Do you prefer watching sports at home on TV or going to a stadium? What are the pros and cons of each, and what was your last experience like?
8. I’d like to know about your favorite and leas